# Investigating sequence motif algorithms #

## PSSM ##

### Part 1. Import in required modules and read in clustal formatted alignments ###

In [19]:
from uqbinfpy import sequence
from uqbinfpy import seqsymbol
from uqbinfpy import prob
from math import log

# Using DNA alphabet
dna = seqsymbol.DNA_Alphabet

# Read in a alignment
aln = sequence.readClustalFile('example.aln', dna)
for seq in aln.seqs:
    print seq

SEQUENCE08: CTCCTTACATGGGC
SEQUENCE05: TGCCAAAAGTGGTC
SEQUENCE04: TGACTATAAAAGGA
SEQUENCE07: CAACTATCTTGGGC
SEQUENCE06: CAACTATCTTGGGC
SEQUENCE01: GACCAAATAAGGCA
SEQUENCE03: TGACTATAAAAGGA
SEQUENCE02: GACCAAATAAGGCA


### Part 2. Calculate the probability for each position/row in the matrix ###

In [20]:
P = prob.IndepJoint([dna for _ in range(aln.alignlen)])
for seq in aln.seqs:
    P.observe(seq)

# See the results
print "The 'probaility matrix'"
P.displayMatrix()
P.displayMatrix(count=True)

The 'probaility matrix'
 		    1	    2	    3	    4	    5	    6	    7	    8	    9	   10	   11	   12	   13	   14
A		0.000	0.500	0.500	0.000	0.375	0.875	0.500	0.375	0.625	0.500	0.250	0.000	0.000	0.500
C		0.375	0.000	0.500	1.000	0.000	0.000	0.000	0.375	0.000	0.000	0.000	0.000	0.250	0.500
G		0.250	0.375	0.000	0.000	0.000	0.000	0.000	0.000	0.125	0.000	0.750	1.000	0.625	0.000
T		0.375	0.125	0.000	0.000	0.625	0.125	0.500	0.250	0.250	0.500	0.000	0.000	0.125	0.000
 		    1	    2	    3	    4	    5	    6	    7	    8	    9	   10	   11	   12	   13	   14
A		    0	    4	    4	    0	    3	    7	    4	    3	    5	    4	    2	    0	    0	    4
C		    3	    0	    4	    8	    0	    0	    0	    3	    0	    0	    0	    0	    2	    4
G		    2	    3	    0	    0	    0	    0	    0	    0	    1	    0	    6	    8	    5	    0
T		    3	    1	    0	    0	    5	    1	    4	    2	    2	    4	    0	    0	    1	    0


### Part 3. Calculate the probability for each position/row in the matrix ###
Notice the '1.0' - setting pseudoconts

In [21]:
P = prob.IndepJoint([dna for _ in range(aln.alignlen)], 1.0)
for seq in aln.seqs:
    P.observe(seq)

# See the results""
print "The 'corrected probability matrix' (with pseudocounts)"
P.displayMatrix()
P.displayMatrix(count=True)

The 'corrected probability matrix' (with pseudocounts)
 		    1	    2	    3	    4	    5	    6	    7	    8	    9	   10	   11	   12	   13	   14
A		0.083	0.417	0.417	0.083	0.333	0.667	0.417	0.333	0.500	0.417	0.250	0.083	0.083	0.417
C		0.333	0.083	0.417	0.750	0.083	0.083	0.083	0.333	0.083	0.083	0.083	0.083	0.250	0.417
G		0.250	0.333	0.083	0.083	0.083	0.083	0.083	0.083	0.167	0.083	0.583	0.750	0.500	0.083
T		0.333	0.167	0.083	0.083	0.500	0.167	0.417	0.250	0.250	0.417	0.083	0.083	0.167	0.083
 		    1	    2	    3	    4	    5	    6	    7	    8	    9	   10	   11	   12	   13	   14
A		    1	    5	    5	    1	    4	    8	    5	    4	    6	    5	    3	    1	    1	    5
C		    4	    1	    5	    9	    1	    1	    1	    4	    1	    1	    1	    1	    3	    5
G		    3	    4	    1	    1	    1	    1	    1	    1	    2	    1	    7	    9	    6	    1
T		    4	    2	    1	    1	    6	    2	    5	    3	    3	    5	    1	    1	    2	    1


### Part 4.1. The PSSM with equiprobable assumption ###

In [22]:
print "The PSSM with equiprobable assumption"
equiprobable = 0.25
print "P(d) = "+str(equiprobable)
matrix = P.getMatrix()
# Manual manipultation. Must be better way
keys = ['A', 'C', 'G', 'T']
for k in keys:
    out = k+" "
    cur = (matrix[k])
    for elem in cur:
        scaled = str(round(log((elem/0.25),2),3))
        if len(scaled) == 3:
            scaled = scaled+"00"
        if len(scaled) == 4:
            scaled = scaled+"0"
        if len(scaled) == 5:
            scaled = " "+scaled
        out = out+"\t"+scaled
    print out

The PSSM with equiprobable assumption
P(d) = 0.25
A 	-1.585	 0.737	 0.737	-1.585	 0.415	 1.415	 0.737	 0.415	 1.000	 0.737	 0.000	-1.585	-1.585	 0.737
C 	 0.415	-1.585	 0.737	 1.585	-1.585	-1.585	-1.585	 0.415	-1.585	-1.585	-1.585	-1.585	 0.000	 0.737
G 	 0.000	 0.415	-1.585	-1.585	-1.585	-1.585	-1.585	-1.585	-0.585	-1.585	 1.222	 1.585	 1.000	-1.585
T 	 0.415	-0.585	-1.585	-1.585	 1.000	-0.585	 0.737	 0.000	 0.000	 0.737	-1.585	-1.585	-0.585	-1.585


### Part 4.2. The PSSM background distribution of all sequences information ###

In [23]:
print "The PSSM background distribution of all sequences information"

# Determine uderlying counts/frequencies
a_cnts = P.getRow('A',count=True)
c_cnts = P.getRow('C',count=True)
g_cnts = P.getRow('G',count=True)
t_cnts = P.getRow('T',count=True)
total = sum(a_cnts)+sum(c_cnts)+sum(g_cnts)+sum(t_cnts)
a_pd = sum(a_cnts)/total
c_pd = sum(c_cnts)/total
g_pd = sum(g_cnts)/total
t_pd = sum(t_cnts)/total

underlying_pd = {'A': a_pd, 'C': c_pd, 'G': g_pd, 'T': t_pd}
print "P(d) = "+str(underlying_pd)

matrix = P.getMatrix()
keys = ['A', 'C', 'G', 'T']
for k in keys:
    out = k+" "
    cur = (matrix[k])
    for elem in cur:
        scaled = str(round(log((elem/underlying_pd[k]),2),3))
        if len(scaled) == 3:
            scaled = scaled+"00"
        if len(scaled) == 4:
            scaled = scaled+"0"
        if len(scaled) == 5:
            scaled = " "+scaled
        out = out+"\t"+scaled
    print out

The PSSM background distribution of all sequences information
P(d) = {'A': 0.32142857142857145, 'C': 0.2261904761904762, 'T': 0.22023809523809523, 'G': 0.23214285714285715}
A 	-1.948	 0.374	 0.374	-1.948	 0.052	 1.052	 0.374	 0.052	 0.637	 0.374	-0.363	-1.948	-1.948	 0.374
C 	 0.559	-1.441	 0.881	 1.729	-1.441	-1.441	-1.441	 0.559	-1.441	-1.441	-1.441	-1.441	 0.144	 0.881
G 	 0.107	 0.522	-1.478	-1.478	-1.478	-1.478	-1.478	-1.478	-0.478	-1.478	 1.329	 1.692	 1.107	-1.478
T 	 0.598	-0.402	-1.402	-1.402	 1.183	-0.402	 0.920	 0.183	 0.183	 0.920	-1.402	-1.402	-0.402	-1.402
